# **Cortex Predicitve Analytics Fundraising Scenario**

**ROUND 2 - STAGE 1: Predict donation amount**

Daniel Salvador Cázares García A01197517

***
# **Set Up**

Run this notebook before 'R2 Probability of Giving'

## **Connect tSPy**

**0. Connect to Google Drive**

In [ ]:
my_folder = "/content/drive/MyDrive/"

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir(my_folder)
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Concentración IA & DS/Reto SAS


**1. Python & Java version**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**3. Configuration file**

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 2
'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create .authinfo**

In [ ]:
%%writefile .authinfo
oda user your-user@mail.com password your-password

Overwriting .authinfo


In [ ]:
!cp .authinfo ~/.authinfo

**5- Establish Connection**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile="/content/drive/MyDrive/sascfg_personal.py")
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 578



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/Concentración IA & DS/Reto SAS/sascfg_personal.py
WORK Path             = /saswork/SAS_workCC610001CA3F_odaws02-usw2-2.oda.sas.com/SAS_work47980001CA3F_odaws02-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 117311


***
## Connect to Cortex Data Sets

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 11:42:00 AM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 11:42:00 AM

33         


### Transform SAS dataset to dataframe

In [ ]:
import pandas as pd

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

In [ ]:
target_data = pd.merge(data1, data2, on=["ID"],how="right")
target_data = target_data.loc[(target_data['GaveThisYear'] ==1)]
target_data.sample(2)

***
# **Model**

## Data Preparation

Missing Values

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe.fit(target_data[['Education', 'City']])
ohe_df = pd.DataFrame(data=ohe.transform(target_data[['Education', 'City']]), columns=ohe.get_feature_names_out())

target_data = pd.concat([target_data.reset_index(drop=True), ohe_df], axis='columns')

from sklearn.preprocessing import LabelEncoder

target_data['Education'] = LabelEncoder().fit_transform(target_data['Education'])
target_data['City'] = LabelEncoder().fit_transform(target_data['City'])

In [ ]:
target_data['Recency'] = target_data['Recency'].fillna((target_data['GaveLastYear'] == 1.0).map({True: 1.0}))
target_data['Recency'] = target_data['Recency'].fillna(value=0)
target_data['Frequency'] = target_data['Frequency'].fillna((target_data['GaveLastYear'] == 1.0).map({True: 1.0}))
target_data['Frequency'] = target_data['Frequency'].fillna(value=0)

In [ ]:
target_data['MinGift'] = target_data['MinGift'].fillna(target_data['AmtLastYear'])  
target_data['MaxGift'] = target_data['MaxGift'].fillna(target_data['AmtLastYear'])
target_data['TotalGift'] = target_data['TotalGift'].fillna(target_data['AmtLastYear'])

## Data Partition

In [ ]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(target_data, train_size=0.8, random_state=42) 

In [ ]:
selected_cols = ['Woman', 'Age', 'Salary',
       'NbActivities', 'Referrals', 'Recency',
       'TotalGift', 'MinGift', 'MaxGift',
       'GaveLastYear', 'AmtLastYear', 'Contact',
       'Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']

In [ ]:
X_train = train[selected_cols] 
Y_train = train['AmtThisYear']

X_valid = validation[selected_cols] 
Y_valid = validation['AmtThisYear']


## Training

In [ ]:
import numpy as np
from sklearn import metrics

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DT_model = DecisionTreeRegressor(max_depth = 5, random_state=42).fit(X_train, Y_train)
DT_predict = DT_model.predict(X_valid)

In [ ]:
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

### XGBoost

In [ ]:
import xgboost as xg

xgb_r = xg.XGBRegressor(objective ='reg:linear')
xgb_r.fit(X_train, Y_train)
xgb_pred = xgb_r.predict(X_valid)

In [ ]:
#MAE
print(metrics.mean_absolute_error(Y_valid, xgb_pred))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid, xgb_pred)))

***
# Results

## Scoring Data

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

## Predict for members who were contacted

### Data

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

In [ ]:

ohe = OneHotEncoder(sparse=False)
ohe.fit(scoring_data_contact[['Education', 'City']])
ohe_df = pd.DataFrame(data=ohe.transform(scoring_data_contact[['Education', 'City']]), columns=ohe.get_feature_names_out())

scoring_data_contact = pd.concat([scoring_data_contact.reset_index(drop=True), ohe_df], axis='columns')


scoring_data_contact['Education'] = LabelEncoder().fit_transform(scoring_data_contact['Education'])
scoring_data_contact['City'] = LabelEncoder().fit_transform(scoring_data_contact['City'])

scoring_data_contact['Recency'] = scoring_data_contact['Recency'].fillna((scoring_data_contact['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data_contact['Recency'] = scoring_data_contact['Recency'].fillna(value=0)
scoring_data_contact['Frequency'] = scoring_data_contact['Frequency'].fillna((scoring_data_contact['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data_contact['Frequency'] = scoring_data_contact['Frequency'].fillna(value=0)

scoring_data_contact['MinGift'] = scoring_data_contact['MinGift'].fillna(scoring_data_contact['AmtLastYear'])  
scoring_data_contact['MaxGift'] = scoring_data_contact['MaxGift'].fillna(scoring_data_contact['AmtLastYear'])
scoring_data_contact['TotalGift'] = scoring_data_contact['TotalGift'].fillna(scoring_data_contact['AmtLastYear'])

In [ ]:
X_contact = scoring_data_contact[selected_cols]
#X_contact = StandardScaler().fit_transform(X_contact)

### Training

In [ ]:
#contact_pred = DT_model.predict(X_contact)
contact_pred = xgb_r.predict(X_contact)

In [ ]:
scoring_data_contact['Prediction'] = contact_pred
scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

## Predict for members who were NOT contacted

### Data

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

In [ ]:

ohe = OneHotEncoder(sparse=False)
ohe.fit(scoring_data_nocontact[['Education', 'City']])
ohe_df = pd.DataFrame(data=ohe.transform(scoring_data_nocontact[['Education', 'City']]), columns=ohe.get_feature_names_out())

scoring_data_nocontact = pd.concat([scoring_data_nocontact.reset_index(drop=True), ohe_df], axis='columns')

scoring_data_nocontact['Education'] = LabelEncoder().fit_transform(scoring_data_nocontact['Education'])
scoring_data_nocontact['City'] = LabelEncoder().fit_transform(scoring_data_nocontact['City'])

scoring_data_nocontact['Recency'] = scoring_data_nocontact['Recency'].fillna((scoring_data_nocontact['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data_nocontact['Recency'] = scoring_data_nocontact['Recency'].fillna(value=0)
scoring_data_nocontact['Frequency'] = scoring_data_nocontact['Frequency'].fillna((scoring_data_nocontact['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data_nocontact['Frequency'] = scoring_data_nocontact['Frequency'].fillna(value=0)

scoring_data_nocontact['MinGift'] = scoring_data_nocontact['MinGift'].fillna(scoring_data_nocontact['AmtLastYear'])  
scoring_data_nocontact['MaxGift'] = scoring_data_nocontact['MaxGift'].fillna(scoring_data_nocontact['AmtLastYear'])
scoring_data_nocontact['TotalGift'] = scoring_data_nocontact['TotalGift'].fillna(scoring_data_nocontact['AmtLastYear'])

In [ ]:
X_nocontact = scoring_data_nocontact[selected_cols]
#X_nocontact = StandardScaler().fit_transform(X_nocontact)

### Training

In [ ]:
#contact_pred = DT_model.predict(X_nocontact)
no_contact_pred = xgb_r.predict(X_nocontact)

In [ ]:
scoring_data_nocontact['Prediction'] = no_contact_pred
scoring_data_nocontact = scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

## Exporting Results to a CSV File

In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(5)

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
!head Round2_Output_amt.csv